# Clasificador CNN a partir de una red pre-entrenada:

In [3]:
import sys
import os
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense, Activation
from keras.layers import  Convolution2D, MaxPooling2D, Dense, GlobalAveragePooling2D
from keras.models import Model
from keras import backend as K
from keras import applications

Using TensorFlow backend.


### Modelo VGG16:

Se carga el modelo VGG16: 

keras.applications.vgg16.VGG16(include_top=True, weights='imagenet', input_tensor=None, input_shape=None, pooling=None, classes=1000).

Este modelo contiene los pesos obtenidos con el pre-entrenamiento en ImageNet. Por defecto, establece una imagen de entrada de 224x224.

In [4]:
vgg = applications.vgg16.VGG16(include_top=False)#Cargamos la red
vgg.summary()#Muestra el contenido de la red.

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
bloc

Se ha pasado como argumento include_top=False. La red vgg16 que aporta keras contiene por defecto una última capa de predicción de mil neuronas (para clasificar mil clases diferentes). La red que se busca solo clasificará tres clases diferentes. Con include_top=False se elimina esta última capa. En la siguiente línea de código se carga la red al completo:

In [5]:
vgg_original = applications.vgg16.VGG16()
vgg_original.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

### Modificación de la última capa:

In [6]:
# add a global spatial average pooling layer
x = vgg.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)
# and a logistic layer -- let's say we have 3 classes
predictions = Dense(3, activation='softmax')(x)
model = Model(inputs=vgg.input, outputs=predictions)

In [7]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0         
__________

### Congelación de capas:

In [8]:
# let's visualize layer names and layer indices to see how many layers
# we should freeze:
for i, layer in enumerate(model.layers):
   print(i, layer.name)

0 input_1
1 block1_conv1
2 block1_conv2
3 block1_pool
4 block2_conv1
5 block2_conv2
6 block2_pool
7 block3_conv1
8 block3_conv2
9 block3_conv3
10 block3_pool
11 block4_conv1
12 block4_conv2
13 block4_conv3
14 block4_pool
15 block5_conv1
16 block5_conv2
17 block5_conv3
18 block5_pool
19 global_average_pooling2d_1
20 dense_1
21 dense_2


In [9]:
for layer in model.layers[:19]:
   layer.trainable = False
for layer in model.layers[19:]:
   layer.trainable = True

In [10]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0         
__________

### Tratamiento de las imágenes:

In [23]:
#Se almacenan en variables los directorios en los que se encuentran las imágenes
data_entrenamiento = './data/entrenamiento'
data_validacion = './data/validacion'

#Parámetros importantes:
epocas=10
longitud, altura = 150, 150
batch_size = 32 #Imágenes a procesar en cada paso
pasos = 10
validation_steps = 30 #Imágenes de validación que se pasan al final de cada época
clases = 3
lr = 0.0004 #Learning rate

flow_from_directory(directory, target_size=(256, 256), color_mode='rgb', classes=None, class_mode='categorical', batch_size=32, shuffle=True, seed=None, save_to_dir=None, save_prefix='', save_format='png', follow_links=False, subset=None, interpolation='nearest')

keras.preprocessing.image.ImageDataGenerator(featurewise_center=False, samplewise_center=False, featurewise_std_normalization=False, samplewise_std_normalization=False, zca_whitening=False, zca_epsilon=1e-06, rotation_range=0, width_shift_range=0.0, height_shift_range=0.0, brightness_range=None, shear_range=0.0, zoom_range=0.0, channel_shift_range=0.0, fill_mode='nearest', cval=0.0, horizontal_flip=False, vertical_flip=False, rescale=None, preprocessing_function=None, data_format=None, validation_split=0.0, dtype=None)

In [24]:
###Procesamiento del conjunto de entrenamieto:
entrenamiento_datagen = ImageDataGenerator(
    rescale=1. / 255, 
    shear_range=0.2, #Inclina las imágenes
    zoom_range=0.2, #Zoom a algunas imágenes
    horizontal_flip=True) #Invierte imágenes para distinguir direcionalidad

###Procesamiento del conjunto de validación:
#No es necesario inclinar, hacer zoom ni invertir las imágenes.
test_datagen = ImageDataGenerator(rescale=1. / 255)

###Generación del conjunto de entrenamieto:
entrenamiento_generador = entrenamiento_datagen.flow_from_directory(
    data_entrenamiento,
    target_size=(altura, longitud),
    batch_size=batch_size,
    class_mode='categorical') #Se busca una clasificación categórica

###Generación del conjunto de validación:
validacion_generador = test_datagen.flow_from_directory(
    data_validacion,
    target_size=(altura, longitud),
    batch_size=batch_size,
    class_mode='categorical')

Found 999 images belonging to 3 classes.
Found 2043 images belonging to 3 classes.


### Definición del modelo CNN: función pérdida y optimizador:

In [25]:
model.compile(loss='categorical_crossentropy',
            optimizer=optimizers.Adam(lr=lr),
            metrics=['accuracy'])

### Entrenamiento del modelo:

fit_generator(generator, steps_per_epoch=None, epochs=1, verbose=1, callbacks=None, validation_data=None, validation_steps=None, validation_freq=1, class_weight=None, max_queue_size=10, workers=1, use_multiprocessing=False, shuffle=True, initial_epoch=0)

In [26]:
model.fit_generator(
    entrenamiento_generador,
    steps_per_epoch=pasos,
    epochs=epocas,
    validation_data=validacion_generador,
    validation_steps=validation_steps)

Epoch 1/10
10/10 [==============================] - 204s 20s/step - loss: 0.4190 - acc: 0.8438 - val_loss: 0.4193 - val_acc: 0.8417
Epoch 2/10
10/10 [==============================] - 198s 20s/step - loss: 0.4559 - acc: 0.8156 - val_loss: 0.3974 - val_acc: 0.8406
Epoch 3/10
10/10 [==============================] - 196s 20s/step - loss: 0.4402 - acc: 0.8375 - val_loss: 0.4071 - val_acc: 0.8524
Epoch 4/10
10/10 [==============================] - 378s 38s/step - loss: 0.3637 - acc: 0.8639 - val_loss: 0.3717 - val_acc: 0.8469
Epoch 5/10
10/10 [==============================] - 420s 42s/step - loss: 0.3903 - acc: 0.8531 - val_loss: 0.3713 - val_acc: 0.8618
Epoch 6/10
10/10 [==============================] - 404s 40s/step - loss: 0.3372 - acc: 0.8750 - val_loss: 0.3366 - val_acc: 0.8635
Epoch 7/10
10/10 [==============================] - 399s 40s/step - loss: 0.3634 - acc: 0.8450 - val_loss: 0.3787 - val_acc: 0.8440
Epoch 8/10
10/10 [==============================] - 408s 41s/step - loss: 0.

### Cálculo de la función pérdida:

El método evaluate_generator(generator, steps=None, callbacks=None, max_queue_size=10, workers=1, use_multiprocessing=False, verbose=0) se encarga de calcular la función de pérdida dados unos datos de entrada y el nivel de aciertos del modelo para una muestra dada.

In [29]:
score = model.evaluate_generator(validacion_generador, steps=20, verbose=1)
print('Test accuracy:', score[1])

20/20 [==============================] - 204s 10s/step
Test accuracy: 0.8671875


In [31]:
print(score)

[0.3152485482394695, 0.8671875]


### Guardar el modelo entrenado:

Para no tener que entrenar la red neuronal creada cada vez que se quiera usar, se crea un archivo donde se guarda el modelo creado, y otro donde se guardan los pesos obtenidos para las neuronas después del entranmiento.

In [32]:
import os 
dir = './vgg16/'
if not os.path.exists(dir):
    os.mkdir(dir)
model.save('./vgg16/modelo_vgg16.h5')#Se guarda la estructura de la cnn
model.save_weights('./vgg16/pesos_vgg16.h5')#Se guardan los pesos de la cnn

### Pruebas de clasificación:

In [34]:
import numpy as np
from keras.preprocessing.image import load_img, img_to_array
from keras.models import load_model

In [74]:
###Función predicción:
def predict(file):
  x = load_img(file, target_size=(longitud, altura))
  x = img_to_array(x)
  x = np.expand_dims(x, axis=0)
  array = model.predict(x)
  print(array)  
  result = array[0]
  print(result)
  answer = np.argmax(result)
  print(answer)  
  if answer == 0:
    print("pred: Perro")
  elif answer == 1:
    print("pred: Gato")
  elif answer == 2:
    print("pred: Gorila")
  return answer

In [89]:
predict('cat.17.jpg')

[[7.0992156e-14 0.0000000e+00 1.0000000e+00]]
[7.0992156e-14 0.0000000e+00 1.0000000e+00]
2
pred: Gorila


2

In [72]:
###Cargar modelo desde el directorio:
carga_modelo = './vgg16/modelo_vgg16.h5'
pesos_modelo = './vgg16/pesos_vgg16.h5'
cnn = load_model(carga_modelo)
cnn.load_weights(pesos_modelo)
cnn.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0         
__________